<a href="https://colab.research.google.com/github/nthanhkhang/vietnamese-pos-tagging/blob/main/Vietnamese_POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1499]:
!pip install pyvi
import string
import numpy
import sklearn.svm as svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
import io
from collections import defaultdict
import random
from pyvi import ViTokenizer, ViPosTagger

In [1500]:
#!wget https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_train.txt
#!wget https://raw.githubusercontent.com/nthanhkhang/Natural-Language-Processing/main/Data/vi_test.txt

In [1501]:
tagsetDict = {"N" : 1,
              "Np" : 2,
              "Nc" : 3,
              "Nu" : 4,
              "V" : 5,
              "A" : 6,
              "P" : 7,
              "L" : 8,
              "M" : 9,
              "R" : 10,
              "E" : 11,
              "C" : 12,
              "I" : 13,
              "T" : 14,
              "B" : 15,
              "Y" : 16,
              "S" : 17,
              "X" : 19 }
inverseTagsetDict = {tagsetDict[k]:k for k in tagsetDict}

In [1502]:
wordBank = defaultdict()
bigramBank = defaultdict()
bigramDict = defaultdict(int)

bigramFreq = {x:[0]*18 for x in tagsetDict}
mostCommonBigrams = defaultdict()

prevParts = None

f_tagged_train = io.open("vi_train.txt", encoding='utf-8').readlines()
f_tagged_test = io.open("vi_test.txt", encoding='utf-8').readlines()

train = f_tagged_train[:8900]
test = f_tagged_test[:10000]
print(train)
print(test)

['Trên/E đường/N đi/V ,/, có/V một/M lần/N xe/N cô/N suýt/R rơi/V xuống/R vực/N ở/E đèo/N Ngoạn_Mục/Np ./.\n', '\n', 'Trong/E một/M trận/N đánh/V ác_liệt/A bên/N thành/N cổ/N Quảng_Trị/Np ,/, một/M loạt/N đạn/N pháo/N của/E kẻ_thù/N đã/R rơi/V trúng/A chỗ/N chiến_sĩ/N thông_tin/N Nguyễn_Văn_Thạc/Np .../... Hôm/N đó/P ngày/N 30/M -/- 7/M -/- 1972/M ./.\n', '\n', 'Sau/E khi/N H./Ny “/“ AK/Ny ”/” bị/V xộ/V khám/V ,/, Hoàng/Np đã/R hoàn_lương/V ,/, bây_giờ/P đang/R phụ/V việc/N cho/E gia_đình/N Quân/Np ./.\n', '\n', 'Hà/Np ,/, 21/M tuổi/N -/- làm/V nghề/N hớt_tóc/V ,/, khai/V :/: “/“ Ban_đầu/N nghe/V mấy/L ảnh/N nói/V thuốc/N lắc/V không/R nghiện/V nên/C em/N uống/V thử/V ./.\n', '\n', 'Theo/E anh/Nc Thông/Np ,/, người/N già/A ở/E Đức/Np thường/R sống/V một_mình/Ny và/C sống/V với/E những/L kỷ_vật/N của/E riêng/A mình/P mang/V về/R từ/E những/L chuyến/N du_lịch/V hay/C những/L tặng_vật/N của/E cả/P một/M đời/N ./.\n', '\n', 'Người/N đến/V muộn/A một_chút/L ,/, phà/N vừa/R rời/V bến/N thì/C

In [1503]:
for line in train:
    l_split = line.split()
    for i,w in enumerate(l_split):
        parts = w.split("/")
        if i >= 1:
            prevParts = l_split[i-1].split("/")
        if len(parts) == 1 or \
            parts[1] not in tagsetDict:
            continue

        if i >= 2 and prevParts[1] in string.punctuation:
            prevParts = l_split[i-1].split("/")

        word = parts[0]
        pos = parts[1]

        if i >= 1:
            prevWord = prevParts[0]
            prevPos = prevParts[1]
            bigramBank[ word ] = (pos, prevWord, prevPos)
            bigramDict[ (pos, prevPos) ] += 1

        if word not in wordBank:
            wordBank[word] = [pos]
        else:
            wordBank[word] += [pos]
print(wordBank)

defaultdict(None, {'Trên': ['E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'A'], 'đường': ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', '

In [1504]:
for k in bigramFreq:
    maxFreq = 0
    maxPos = "X"
    for i,x in enumerate(bigramFreq[k]):
        if x > maxFreq:
            maxFreq = x
            maxPos = inverseTagsetDict[i]
    mostCommonBigrams[k] = maxPos
print ("finished getting training data")

finished getting training data


In [1505]:
def feature(word,wordIdx, lineSize, line):
    feat = [1]
    sentPercent = float(wordIdx)/float(lineSize)
    feat.append(sentPercent)

    if word[0].isupper() and wordIdx != 0:
        feat.append(1)
    else:
        feat.append(0)

    posIdx_array = ([0] * len(tagsetDict))
    posSet = []
    if word in wordBank:
        posSet = wordBank[word]
    else:
        if wordIdx == 0:
            posSet = list(tagsetDict.keys())[0] 
            posIdx_array[tagsetDict[posSet]] = 1
            return feat + posIdx_array + [0]
        else:
            prevWord = line[wordIdx-1]
            if prevWord in wordBank:
                prevPos = wordBank[prevWord]
                maxPos = mostCommonBigrams[prevPos]
                if prevPos == "E":
                    maxPos = 3
                posIdx_array[tagsetDict[maxPos]] = 1
                feat += posIdx_array + [tagsetDict[maxPos]]
                return feat
            else:
                posSet = list(tagsetDict.keys())[0]  
                posIdx_array[tagsetDict[posSet]] = 1
                feat += posIdx_array + [0]
                return feat

    for pos in posSet:
        posIdx = tagsetDict[pos]
        posIdx_array[posIdx] += 1.0/len(wordBank[word])
    feat += (posIdx_array) + [0]
    return feat

In [1506]:
y = []
X_train = []
for line in train:
    l_split = line.split()
    for i,w in enumerate(l_split):
        parts = w.split("/")
        word = parts[0]
        len_line = len(l_split)

        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue
        y.append(wordBank[word][0])
        X_train.append(feature(word,i,len_line, l_split))
print ("finished creating y and xtrain")

finished creating y and xtrain


In [1507]:
print ( len(X_train) )
print ( len(y) )
print(X_train[0:10])
print(y[0:10])
train_fit = OneVsRestClassifier(LinearSVC(random_state=0)).fit(X_train,y)

84883
84883
[[1, 0.0, 0, 0, 0, 0, 0, 0, 0, 0.07142857142857142, 0, 0, 0, 0, 0.9285714285714283, 0, 0, 0, 0, 0, 0, 0], [1, 0.058823529411764705, 0, 0, 0.9999999999999968, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.11764705882352941, 0, 0, 0, 0, 0, 0, 0.9032258064516225, 0, 0, 0, 0, 0.08918406072106257, 0, 0, 0, 0.007590132827324478, 0, 0, 0, 0], [1, 0.23529411764705882, 0, 0, 0, 0, 0, 0, 0.993111366245681, 0, 0, 0, 0, 0.003444316877152698, 0, 0, 0, 0.003444316877152698, 0, 0, 0, 0], [1, 0.29411764705882354, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.9999999999999745, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.35294117647058826, 0, 0, 0.9679487179487155, 0, 0, 0, 0.03205128205128205, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.4117647058823529, 0, 0, 0.988372093023257, 0, 0, 0, 0.011627906976744186, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.47058823529411764, 0, 0, 0.7258883248730975, 0, 0.2741116751269033, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.5294117647058824, 0, 0, 0, 0

In [1508]:
X_test = []
correct_results = []
for line in test:
    l_split = line.split()
    for i,w in enumerate(l_split):
        len_line = len(l_split)
        parts = w.split("/")
        if len(parts) == 1 or parts[1] not in tagsetDict:
            continue
        word = parts[0]
        pos = parts[1]
        if word in string.punctuation or word == ":.":
            continue
        X_test.append(feature(word,i,len_line,l_split))
        correct_results.append(pos)
print("finished getting testing data")
print(X_test)

finished getting testing data
[[1, 0.0, 0, 0, 0, 0, 0, 0, 0, 0.07142857142857142, 0, 0, 0, 0, 0.9285714285714283, 0, 0, 0, 0, 0, 0, 0], [1, 0.038461538461538464, 0, 0, 0.9999999999999968, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.07692307692307693, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.11538461538461539, 0, 0, 0, 0, 0, 0, 0, 0.9999999999999978, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.15384615384615385, 0, 0, 0.49999999999999994, 0, 0, 0, 0.1, 0, 0, 0, 0.39999999999999997, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.19230769230769232, 0, 0, 0.9999999999999996, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.2692307692307692, 0, 0, 0.4856230031948866, 0, 0.5143769968051104, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.34615384615384615, 0, 0, 0.9999999999999986, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0.4230769230769231, 0, 0, 0.4856230031948866, 0, 0.5143769968051104, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [

In [1509]:
predicted_results = train_fit.predict(X_test)
print("Predicted Results:",predicted_results[:10])
print("Correct Results  :",correct_results[:10])

numCorrect = 0
for cor,pred in zip(correct_results, predicted_results):
    if cor == pred:
        numCorrect += 1

accuracy = 1.0*numCorrect/len(correct_results)
print("Accuracy:",accuracy*100)

Predicted Results: ['E' 'N' 'V' 'A' 'N' 'N' 'N' 'N' 'N' 'C']
Correct Results  : ['E', 'N', 'V', 'A', 'N', 'N', 'N', 'N', 'N', 'C']
Accuracy: 86.69771726891187


In [1510]:
def toString(wl):
  wl=longest_matching(wl, bi_grams, tri_grams)
  X=[]
  A=[]
  text=""
  for i in set(wl):
    if i ==".":
      A="./."
    else:
      X.append(feature(i,1,1,1))
      A=str(train_fit.predict(X))
  #print(A)  
  for i in range(len(wl)):
    text+=wl[i]
    text+='/'
    text+=str(A[2])
    text+=' '
  return text

In [1511]:
if __name__ == "__main__":
    wr ="Dù khá đắt nhưng tôi vẫn đồng ý."
    wl = ViTokenizer.tokenize(wr)
    print(wr)
    wl = wl.split()  
    for i in wl:
      print(toString(i),end='')

Dù khá đắt nhưng tôi vẫn đồng ý.
Dù/C khá/R đắt/A nhưng/C tôi/P vẫn/R đồng_ý/V ./. 